In [ ]:
# === Metoda 1: k-Nearest Neighbors (k-NN) “od zera” z 5-fold CV, 4 wartościami k i własnym paskiem postępu ===

import pandas as pd
import numpy as np
from collections import Counter
import sys
import time

# 1) Wczytanie i przygotowanie danych
df = pd.read_csv(r'C:\Users\Kasia\Desktop\train_data.csv', sep=';').dropna()
for col in df.select_dtypes(include=['object']).columns:
    df[col], _ = pd.factorize(df[col])

# Subsampling dla wydajności (10 000 próbek)
np.random.seed(0)
idx = np.random.choice(len(df), size=10000, replace=False)
df = df.iloc[idx].reset_index(drop=True)
X = df.drop('Stay', axis=1).values
y = df['Stay'].values

# 2) Stratified 5-fold CV
def make_folds(y, k=5, seed=0):
    np.random.seed(seed)
    folds = [[] for _ in range(k)]
    for cls in np.unique(y):
        cls_idx = np.where(y == cls)[0]
        np.random.shuffle(cls_idx)
        for i, ix in enumerate(cls_idx):
            folds[i % k].append(ix)
    return [np.array(f) for f in folds]

folds = make_folds(y, k=5)

# 3) Implementacja k-NN “od zera”
def make_knn(X_tr, y_tr, k, chunk_size=200):
    def predict(Xm):
        preds = np.empty(len(Xm), dtype=y_tr.dtype)
        for start in range(0, len(Xm), chunk_size):
            end   = min(start + chunk_size, len(Xm))
            batch = Xm[start:end]
            D     = np.linalg.norm(batch[:, None, :] - X_tr[None, :, :], axis=2)
            nn    = np.argpartition(D, k, axis=1)[:, :k]
            for i, neigh in enumerate(nn):
                preds[start + i] = Counter(y_tr[neigh]).most_common(1)[0][0]
        return preds
    return predict

# 4) Ewaluacja z paskami postępu
def eval_k(k):
    train_scores, test_scores = [], []
    total = len(folds)
    for i in range(total):
        # pokazujemy postęp foldów
        pct = (i+1)/total*100
        print(f'\r[k={k}] Fold {i+1}/{total} ({pct:.1f}%)', end='', flush=True)
        
        te = folds[i]
        tr = np.hstack([folds[j] for j in range(total) if j != i])
        X_tr, y_tr = X[tr], y[tr]
        X_te, y_te = X[te], y[te]
        clf = make_knn(X_tr, y_tr, k)
        train_scores.append((clf(X_tr) == y_tr).mean())
        test_scores .append((clf(X_te) == y_te).mean())
    print()  # newline po foldach
    return np.mean(train_scores), np.mean(test_scores)

# 5) Pętla po wartościach k z głównym paskiem postępu
k_values = [1, 3, 5, 7]
results = {}
total_k = len(k_values)
for idx, k in enumerate(k_values):
    pct_k = (idx+1)/total_k*100
    print(f'\rPrzetwarzanie k={k} ({pct_k:.1f}% wszystkich)', end='', flush=True)
    results[k] = eval_k(k)
print()  # newline po wszystkich k

# 6) Wyświetlenie wyników
print("\nk-NN (parametr k) – 5-fold CV:")
for k, (tr, te) in results.items():
    print(f"  k = {k:2d} → train_acc = {tr:.3f}, test_acc = {te:.3f}")


# WNIOSKI
# k = 1: train_acc = 1.000 → model overfituje (każdy punkt jest swoim sąsiadem), test_acc = 0.201 → bardzo słaba generalizacja
# k = 3: train_acc = 0.648 → mniejszy overfitting, test_acc = 0.205 → niewielka poprawa (+0.4 p.p.)
# k = 5: train_acc = 0.489 → rośnie bias, test_acc = 0.229 → wyraźna poprawa (+2.8 p.p.)
# k = 7: train_acc = 0.446 → najwyższy bias, test_acc = 0.240 → najlepszy balans bias/variance w badanym zakresie

# Analiza wykazała, że przy małych wartościach k (np. 1) model silnie overfituje, osiągając 100 % na treningu, ale poniżej 21 % na 
# danych walidacyjnych. Z kolei większe k (5–7) zmniejszają wariancję kosztem większego biasu, poprawiając test_acc do blisko 24 %. 
# Optymalny kompromis między biasem a wariancją uzyskano dla k = 7, co skutkuje najlepszą generalizacją na analizowanym zbiorze.


[k=1] Fold 5/5 (100.0%)% wszystkich)
[k=3] Fold 5/5 (100.0%)% wszystkich)
[k=5] Fold 5/5 (100.0%)% wszystkich)
[k=7] Fold 5/5 (100.0%)0% wszystkich)


k-NN (parametr k) – 5-fold CV:
  k =  1 → train_acc = 1.000, test_acc = 0.201
  k =  3 → train_acc = 0.648, test_acc = 0.205
  k =  5 → train_acc = 0.489, test_acc = 0.229
  k =  7 → train_acc = 0.446, test_acc = 0.240


In [ ]:
# === Metoda 2: Nearest Centroid “od zera” z 5-fold CV i 4 metrykami ===

import numpy as np

def make_centroid(X_tr, y_tr, metric):
    classes = np.unique(y_tr)
    cents = {c: X_tr[y_tr == c].mean(axis=0) for c in classes}
    def dist(a, b):
        if metric == 'euclidean':
            return np.linalg.norm(a - b)
        if metric == 'manhattan':
            return np.sum(np.abs(a - b))
        if metric == 'chebyshev':
            return np.max(np.abs(a - b))
        # cosine
        return 1 - (a @ b) / (np.linalg.norm(a) * np.linalg.norm(b))
    def predict(Xm):
        preds = np.empty(len(Xm), dtype=y_tr.dtype)
        for i, x in enumerate(Xm):
            dists = [dist(x, cents[c]) for c in classes]
            preds[i] = classes[np.argmin(dists)]
        return preds
    return predict

def eval_metric(metric):
    train_scores, test_scores = [], []
    for i in range(len(folds)):
        tr_idx = np.hstack([folds[j] for j in range(len(folds)) if j != i])
        te_idx = folds[i]
        X_tr, y_tr = X[tr_idx], y[tr_idx]
        X_te, y_te = X[te_idx], y[te_idx]
        clf = make_centroid(X_tr, y_tr, metric)
        train_scores.append((clf(X_tr) == y_tr).mean())
        test_scores .append((clf(X_te) == y_te).mean())
    return np.mean(train_scores), np.mean(test_scores)

metrics = ['euclidean', 'manhattan', 'chebyshev', 'cosine']
results = {m: eval_metric(m) for m in metrics}

print("Nearest Centroid – 5-fold CV:")
for m, (tr, te) in results.items():
    print(f"  {m:<10} → train_acc = {tr:.3f}, test_acc = {te:.3f}")

# WNIOSKI
# euclidean : train_acc ≈0.054, test_acc ≈0.054 → centroid jest zbyt uproszczony, model nie uchwycił struktury danych
# manhattan : train_acc ≈0.058, test_acc ≈0.058 → niewielka poprawa względem euklidesowej, ale nadal bardzo słabo
# chebyshev : train_acc ≈0.049, test_acc ≈0.049 → najgorszy wynik, metryka Chebysheva nieadekwatna dla tych cech
# cosine    : train_acc ≈0.045, test_acc ≈0.045 → odległość kosinusowa również nie oddaje relacji między próbkami

# Centroidy oparte na średnich punktów każdej klasy są tutaj zdecydowanie zbyt uproszczone, co prowadzi do bardzo niskich 
# accuracy (~5 %) niezależnie od metryki. Metryka Manhattan wypadła nieznacznie lepiej, ale nadal nie zapewnia zadowalającej 
# generalizacji. Najlepszy sposób na poprawę wyników to zastosowanie bardziej zaawansowanych modeli lub inżynierii cech, ponieważ 
# proste podejście centroidowe nie wystarcza.


Nearest Centroid – 5-fold CV:
  euclidean  → train_acc = 0.059, test_acc = 0.058
  manhattan  → train_acc = 0.059, test_acc = 0.058
  chebyshev  → train_acc = 0.060, test_acc = 0.059
  cosine     → train_acc = 0.053, test_acc = 0.051


In [ ]:
# === Metoda 3: Gaussian Naive Bayes “od zera” z 5-fold CV i 4 wartościami var_smoothing ===

import numpy as np
import math

def make_gnb(X_tr, y_tr, var_smooth):
    classes = np.unique(y_tr)
    stats = {}
    for c in classes:
        Xc = X_tr[y_tr == c]
        mean = Xc.mean(axis=0)
        var  = Xc.var(axis=0) + var_smooth
        stats[c] = (mean, var)
    priors = {c: np.mean(y_tr == c) for c in classes}
    
    def predict(Xm):
        preds = np.empty(len(Xm), dtype=y_tr.dtype)
        for i, x in enumerate(Xm):
            posteriors = {}
            for c, (mean, var) in stats.items():
                # log prior
                logp = math.log(priors[c])
                # log-likelihood of Gaussian
                log_lik = -0.5 * np.sum(np.log(2 * math.pi * var) + ((x - mean) ** 2) / var)
                posteriors[c] = logp + log_lik
            preds[i] = max(posteriors, key=posteriors.get)
        return preds
    
    return predict

def eval_gnb(var_smooth):
    train_scores, test_scores = [], []
    for i in range(len(folds)):
        tr_idx = np.hstack([folds[j] for j in range(len(folds)) if j != i])
        te_idx = folds[i]
        X_tr, y_tr = X[tr_idx], y[tr_idx]
        X_te, y_te = X[te_idx], y[te_idx]
        
        clf = make_gnb(X_tr, y_tr, var_smooth)
        train_scores.append((clf(X_tr) == y_tr).mean())
        test_scores .append((clf(X_te) == y_te).mean())
    return np.mean(train_scores), np.mean(test_scores)

var_smooth_values = [1e-9, 1e-8, 1e-7, 1e-6]
results = {vs: eval_gnb(vs) for vs in var_smooth_values}

print("Gaussian Naive Bayes – 5-fold CV dla różnych var_smoothing:")
for vs, (tr, te) in results.items():
    print(f"  var_smoothing = {vs:.0e} → train_acc = {tr:.3f}, test_acc = {te:.3f}")

# WNIOSKI
# var_smoothing = 1e-9 → train_acc = 0.369, test_acc = 0.359  
#   Minimalne wygładzenie; model jest stabilny, ale nadal osiąga niski wynik (~36%).
# var_smoothing = 1e-8 → train_acc = 0.369, test_acc = 0.359  
#   Dziesięciokrotnie większe wygładzenie nie zmienia accuracy – model jest już niewrażliwy na tak małe zmiany.
# var_smoothing = 1e-7 → train_acc = 0.369, test_acc = 0.359  
#   Kolejne zwiększenie wygładzenia również nie wpływa na wyniki – zakres parametrów za mały, by odcisnąć efekt.
# var_smoothing = 1e-6 → train_acc = 0.369, test_acc = 0.359  
#   Nawet przy największym testowanym wygładzeniu wyniki pozostają identyczne – var_smoothing nie ma znaczenia.

# Model Gaussian Naive Bayes jest całkowicie niewrażliwy na dobór parametru var_smoothing w badanym zakresie, co 
# skazuje, że wariancje cech dominują nad drobnymi poprawkami wygładzenia. Osiągnięte ~36 % accuracy przewyższa 
# proste metody odległościowe, ale nadal pozostaje poniżej użytecznego poziomu. Aby poprawić wyniki, warto wdrożyć 
# zaawansowaną inżynierię cech, modele ensemble lub bardziej złożone klasyfikatory nieliniowe.


Gaussian Naive Bayes – 5-fold CV dla różnych var_smoothing:
  var_smoothing = 1e-09 → train_acc = 0.369, test_acc = 0.359
  var_smoothing = 1e-08 → train_acc = 0.369, test_acc = 0.359
  var_smoothing = 1e-07 → train_acc = 0.369, test_acc = 0.359
  var_smoothing = 1e-06 → train_acc = 0.369, test_acc = 0.359


In [ ]:
# === Metoda 3 (zaawansowana): Gaussian Naive Bayes z PCA – wpływ liczby komponentów ===

import numpy as np
import math

# 1) Funkcje PCA “od zera”
def compute_pca(X, n_components):
    mu = X.mean(axis=0)
    Xc = X - mu
    cov = np.cov(Xc, rowvar=False)
    eigvals, eigvecs = np.linalg.eigh(cov)
    idx = np.argsort(eigvals)[::-1][:n_components]
    comps = eigvecs[:, idx]
    return mu, comps

def transform_pca(X, mu, comps):
    return (X - mu).dot(comps)

# 2) Gaussian NB z opcjonalnym PCA
def make_gnb_pca(X_tr, y_tr, n_comp, var_smooth=1e-9):
    # obliczamy PCA na danych uczących
    mu, comps = compute_pca(X_tr, n_comp)
    Xp = transform_pca(X_tr, mu, comps)
    # zbieramy statystyki: mean, var dla każdej klasy
    classes = np.unique(y_tr)
    stats = {
        c: (
            Xp[y_tr==c].mean(axis=0),
            Xp[y_tr==c].var(axis=0) + var_smooth
        )
        for c in classes
    }
    priors = {c: np.mean(y_tr==c) for c in classes}
    
    def predict(Xm):
        Xm_p = transform_pca(Xm, mu, comps)
        preds = np.empty(len(Xm_p), dtype=y_tr.dtype)
        for i, x in enumerate(Xm_p):
            post = {}
            for c, (m, v) in stats.items():
                # log prior + log-likelihood Gaussa
                logp = math.log(priors[c]) - 0.5 * np.sum(
                    np.log(2*math.pi*v) + (x-m)**2 / v
                )
                post[c] = logp
            preds[i] = max(post, key=post.get)
        return preds
    
    return predict

# 3) Przygotowanie par indeksów 5-fold CV
folds_cv = [
    (
        np.hstack([folds[j] for j in range(len(folds)) if j != i]),
        folds[i]
    )
    for i in range(len(folds))
]

# 4) Ewaluacja dla różnych liczby komponentów PCA
components = [5, 10, 15, X.shape[1]]
results = {}
for n_comp in components:
    train_accs, test_accs = [], []
    for tr_idx, te_idx in folds_cv:
        X_tr, y_tr = X[tr_idx], y[tr_idx]
        X_te, y_te = X[te_idx], y[te_idx]
        clf = make_gnb_pca(X_tr, y_tr, n_comp)
        train_accs.append((clf(X_tr) == y_tr).mean())
        test_accs .append((clf(X_te) == y_te).mean())
    results[n_comp] = (np.mean(train_accs), np.mean(test_accs))

# 5) Wyświetlenie wyników
print("GNB + PCA – 5-fold CV dla różnych n_components:")
for n_comp, (tr, te) in results.items():
    print(f"  n_components = {n_comp:2d} → train_acc = {tr:.3f}, test_acc = {te:.3f}")

# WNIOSKI
# Przy bardzo redukowanej przestrzeni (5 komponentów) model traci dużo informacji, co skutkuje niskim test_acc ≈ 28.6%.
# Zwiększenie liczby komponentów do 10 poprawia zarówno train_acc, jak i test_acc (do ≈ 31.3%).
# Optymalny kompromis uzyskujemy przy 15 komponentach: train_acc ≈ 36.1%, test_acc ≈ 35.2%; użycie wszystkich 17 cech nie przynosi dalszej poprawy.
# W porównaniu do poprzedniej metody bez PCA (var_smoothing), która osiągała test_acc ≈ 36%, stosowanie PCA z 15 komponentami daje niemal równorzędne wyniki,
# ale pozwala redukować wymiarowość i potencjalnie przyspiesza obliczenia w dalszych krokach analizy.


GNB + PCA – 5-fold CV dla różnych n_components:
  n_components =  5 → train_acc = 0.288, test_acc = 0.286
  n_components = 10 → train_acc = 0.323, test_acc = 0.313
  n_components = 15 → train_acc = 0.361, test_acc = 0.352
  n_components = 17 → train_acc = 0.362, test_acc = 0.350


In [ ]:
# === Metoda 4: Softmax Regression (wieloklasowa regresja logistyczna) “od zera” z 5-fold CV i 4 wartościami regularyzacji L2 ===

import numpy as np

# 1) Budowa klasyfikatora Softmax Regression
def make_logreg(X_tr, y_tr, reg, lr=0.1, epochs=200):
    n, d = X_tr.shape
    classes = np.unique(y_tr)
    C = len(classes)
    # inicjalizacja wag i biasów
    W = np.zeros((d, C))
    b = np.zeros(C)
    # mapowanie etykiet na indeksy 0..C-1
    class_to_idx = {c: i for i, c in enumerate(classes)}
    y_idx = np.array([class_to_idx[c] for c in y_tr])
    # one-hot
    Y = np.eye(C)[y_idx]
    # gradient descent
    for _ in range(epochs):
        scores = X_tr.dot(W) + b                          # (n, C)
        exp_s = np.exp(scores - np.max(scores, axis=1, keepdims=True))
        P = exp_s / exp_s.sum(axis=1, keepdims=True)      # (n, C)
        # gradient
        dW = (X_tr.T.dot(P - Y)) / n + reg * W             # (d, C)
        db = (P - Y).mean(axis=0)                          # (C,)
        # update
        W -= lr * dW
        b -= lr * db
    # funkcja predykcji
    def predict(Xm):
        out = Xm.dot(W) + b
        return classes[np.argmax(out, axis=1)]
    return predict

# 2) Funkcja ewaluacji dla danego reg w 5-fold CV
def eval_reg(reg):
    train_scores, test_scores = [], []
    for i in range(len(folds)):
        tr_idx = np.hstack([folds[j] for j in range(len(folds)) if j != i])
        te_idx = folds[i]
        X_tr, y_tr = X[tr_idx], y[tr_idx]
        X_te, y_te = X[te_idx], y[te_idx]
        clf = make_logreg(X_tr, y_tr, reg)
        train_scores.append((clf(X_tr) == y_tr).mean())
        test_scores .append((clf(X_te) == y_te).mean())
    return np.mean(train_scores), np.mean(test_scores)

# 3) Testujemy 4 wartości regularyzacji L2
regs = [0.0, 0.01, 0.1, 1.0]
results = {r: eval_reg(r) for r in regs}

# 4) Wyświetlenie wyników
print("Softmax Regression – 5-fold CV dla różnych L2 regularization:")
for r, (tr, te) in results.items():
    print(f"  reg = {r:.2f} → train_acc = {tr:.3f}, test_acc = {te:.3f}")

# WNIOSKI
# reg = 0.00 → train_acc = 0.207, test_acc = 0.207  
#   Brak regularyzacji daje najlepsze wyniki, model nie wykazuje nadmiernego overfittingu.
# reg = 0.01 → train_acc = 0.200, test_acc = 0.200  
#   Lekka regularyzacja obniża accuracy o ~0.7 p.p., sugerując niewielki wpływ.
# reg = 0.10 → train_acc = 0.173, test_acc = 0.172  
#   Silniejsza regularyzacja prowadzi do underfittingu i znacznego spadku dokładności.
# reg = 1.00 → train_acc = 0.198, test_acc = 0.198  
#   Bardzo duże L2 częściowo przywraca dopasowanie, lecz nadal nie dorównuje modelowi bez regularyzacji.

# Softmax regression osiąga jedynie około 20 % accuracy niezależnie od poziomu L2, co świadczy o tym, że 
# liniowe granice decyzyjne są zbyt proste dla tych danych. Brak regularyzacji daje najlepsze rezultaty, co 
# oznacza, że model nie overfituje znacząco, ale też nie potrafi efektywnie uogólniać. Aby uzyskać sensowną 
# poprawę wyników, warto sięgnąć po bardziej złożone klasyfikatory lub zaawansowaną inżynierię cech.


Softmax Regression – 5-fold CV dla różnych L2 regularization:
  reg = 0.00 → train_acc = 0.207, test_acc = 0.207
  reg = 0.01 → train_acc = 0.200, test_acc = 0.200
  reg = 0.10 → train_acc = 0.173, test_acc = 0.172
  reg = 1.00 → train_acc = 0.198, test_acc = 0.198


In [ ]:
# === Metoda 5: Random Forest – 5-fold CV dla różnych n_estimators (bez tqdm_notebook) ===

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

# 1) Przygotowanie par (train_idx, test_idx) dla 5-fold CV
cv_pairs = [
    (
        np.hstack([folds[j] for j in range(len(folds)) if j != i]),
        folds[i]
    )
    for i in range(len(folds))
]

# 2) Testujemy różne liczby drzew w lesie
n_trees = [10, 50, 100, 200]
results = {}
for n in n_trees:
    model = RandomForestClassifier(
        n_estimators=n,
        random_state=0,
        n_jobs=-1
    )
    res = cross_validate(
        model, X, y,
        cv=cv_pairs,
        scoring='accuracy',
        return_train_score=True,
        n_jobs=-1
    )
    results[n] = (res['train_score'].mean(), res['test_score'].mean())

# 3) Wyświetlenie wyników
print("Random Forest – 5-fold CV dla różnych n_estimators:")
for n, (tr, te) in results.items():
    print(f"  n_estimators = {n:3d} → train_acc = {tr:.3f}, test_acc = {te:.3f}")

# WNIOSKI
# n_estimators =  10 → train_acc = 0.987, test_acc = 0.340
#   Model jest bliski pełnego dopasowania na treningu, ale generalizacja jest niska.
# n_estimators =  50 → train_acc = 1.000, test_acc = 0.379
#   Więcej drzew poprawia test_acc o prawie 4 p.p., zmniejszając wariancję.
# n_estimators = 100 → train_acc = 1.000, test_acc = 0.391
#   Optymalna liczba drzew w badanym zakresie — najlepszy test_acc (~39.1%).
# n_estimators = 200 → train_acc = 1.000, test_acc = 0.389
#   Dalszy wzrost liczby drzew nie przynosi poprawy, a nawet lekko pogarsza generalizację (efekt malejących korzyści).

# Zwiększenie liczby drzew w lesie prowadzi do lepszego uogólnienia – przejście od 10 do 100 drzew 
# podniosło testową dokładność z 34,0 % do 39,1 %. Jednocześnie nawet przy 10 drzew model osiąga niemal 
# perfekcyjne dopasowanie na treningu, co wskazuje na dużą wariancję w modelu. Powyżej ~100 drzew korzyści 
# zaczynają maleć, a dalsze zwiększanie liczby drzew nie poprawia, a wręcz nieznacznie obniża test_acc.


Random Forest – 5-fold CV dla różnych n_estimators:
  n_estimators =  10 → train_acc = 0.987, test_acc = 0.340
  n_estimators =  50 → train_acc = 1.000, test_acc = 0.379
  n_estimators = 100 → train_acc = 1.000, test_acc = 0.391
  n_estimators = 200 → train_acc = 1.000, test_acc = 0.389
